# Challenge: Promotions

In this challenge, you'll develop codes to parse and analyze data returned from another API on Zalando such as [Promos homme (Men's Promotions)
](https://www.zalando.fr/promo-homme/) or [Promos femme (Women's Promotions)](https://www.zalando.fr/promo-femme/). The workflow is almost the same as in the guided lesson but you'll work with different data.

## Obtaining the link

Wrote your codes in the cell below to obtain the data from the API endpoint you choose. A recap of the workflow:

1. Examine the webpages and choose one that you want to work with.

1. Use Google Chrome's DevTools to inspect the XHR network requests. Find out the API endpoint that serves data to the webpage.

1. Test the API endpoint in the browser to verify its data.

1. Change the page number offset of the API URL to test if it's working.

In [ ]:
url='https://www.zalando.fr/api/catalog/articles?categories=promo-homme&limit=84&offset=84&sort=sale'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}

## Reading the data

In the next cell, use Python to obtain data from the API endpoint you chose in the previous step. Workflow:

1. Import libraries.

1. Define the initial API endpoint URL.

1. Make request to obtain data of the 1st page. Flatten the data and store it in an empty object variable.

1. Find out the total page count in the 1st page data.

1. Use a FOR loop to make requests for the additional pages from 2 to page count. Append the data of each additional page to the flatterned data object.

1. Print and review the data you obtained.

In [ ]:
# your code here
import json
import requests
import pandas as pd
from pandas.io.json import json_normalize

response = requests.get(url, headers=headers)
response.status_code

results = response.json()
results


flattened_data = json_normalize(results)

flattened_data1 = json_normalize(flattened_data.articles[0])
flattened_data1

In [ ]:
# Get the total number of pages
total_pages=results['pagination']['page_count']

#cambiamos el offset a 168 para situarnos en la hoja2, ya que cada hoja consta de 84 productos

# Your code
df=pd.DataFrame()
for i in range(total_pages):
    k=84*i
    url=f'https://www.zalando.fr/api/catalog/articles?categories=promo-homme&limit=84&offset=168&sort=sale'
    
    response = requests.get(url, headers=headers)
    results = response.json()
    flattened_data = json_normalize(results)
    flattened_data1 = json_normalize(flattened_data.articles[0])
    flattened_data1=flattened_data1.set_index('sku')
    df = df.append(flattened_data1)

df

## Bonus

Extract the following information from the data:

* The trending brand.

* The product(s) with the highest discount.

* The sum of discounts of all goods (sum_discounted_prices divided by sum_original_prices).

In [ ]:
#trending brand
df['brand_name'].value_counts().index[0]

In [ ]:
#Vemos como vienen los datos de las columnas
set(df['price.original'])
set(df['price.promotional'])

In [ ]:
# extraemos los valores que nos interesan y quitamos la basura
df['price.original']=df['price.original'].str.extract('(\d*,\d*)')
df['price.promotional']=df['price.promotional'].str.extract('(\d*,\d*)')


In [ ]:
# convertimos a tipo string para poder aplicar el replace del ',' al '.'
df['price.promotional'] = df['price.promotional'].astype(str)
df['price.promotional'] = [x.replace(',', '.') for x in df['price.promotional']]

In [ ]:
df['price.original'] = df['price.original'].astype(str)
df['price.original'] = [x.replace(',', '.') for x in df['price.original']]

In [ ]:
# convertimos a float
df['price.promotional'] = df['price.promotional'].astype(float)
df['price.original'] = df['price.original'].astype(float)

In [ ]:
# restamos el precio d descuento al original para ver cuanto es el cobro real.
df['descuento_real'] = df['price.original'] - df['price.promotional']
df1=df.copy()

In [ ]:
# agrupamos el data frame por la marca y se hace la suma de todos las marcas para ver cual es la que hizo los mayores descuento, y hacemos los acomodamos de mayor a menor para poder tomar el primer valor. 
dec_total=df1.groupby(['brand_name']).sum().descuento_real
dec_total.sort_values(ascending=False).index[0]


In [ ]:
#Filtramos por el valor si este fue de cero, entonces no dio descuentos
dec_total[dec_total==0]